## Evaluating Hybrid model

In [ ]:
# imports
import jax
import jax.numpy as jnp

# import jax_cfd.base.equations as eqtns
# import jax_cfd.base as cfd
# import jax_cfd.base.grids as grids

# import jax_cfd.base.advection as advection
# import jax_cfd.base.diffusion as diffusion
# from jax_cfd.base import finite_differences as fd

import numpy as np #?

import xarray
import seaborn
import pickle

import haiku as hk

import matplotlib.pyplot as plt

import jax_cfd.ml.newSaveObject as saving
import jax_cfd.ml.diego_towers as mytowers

In [10]:
path = "./sims_50_seconds/dns_{:.0f}x{:.0f}.nc".format(1024,256)


In [11]:
ds = xarray.open_dataset(path, chunks={'time': '100MB'})

NameError: name 'xarray' is not defined

In [103]:
def sampling(data,factor):
    return data[:,0::factor,0::factor]

In [97]:
def mse(target, ref):
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)
    err = np.sum((target_data - ref_data) ** 2)

    err /= float(target_data.shape[0] * target_data.shape[1])
    return err

In [92]:
def extractData(ds,factor=None):
    u = ds.u.values
    v = ds.v.values
    
    if factor is not None:
        u = sampling(u,factor)
        v = sampling(v,factor)
    
    out = []
    times,_,_ = np.shape(u)
    for time in range(times):
        
        out.append(np.stack([u[time,:,:],v[time,:,:]]))
    
    
    return out


    
out = extractData(ds_coarse,)
# extractData(ds_fine,factor=factor)

In [190]:
def calculateErrors(reference,referenceFactor,otherSim,otherFactor):
    reference = extractData(reference,referenceFactor)
    otherSim = extractData(otherSim,otherFactor)
    print(np.shape(reference))
    (times,_,_,_) = np.shape(reference)
    errors = []
    for time in range(times):
        errors.append(mse(reference[time][0,:,:],otherSim[time][0,:,:]))
    
    return errors

errors_LI = calculateErrors(ds_fine_1080,16,ds_coarse_corrected,None)
errors_DNS = calculateErrors(ds_fine_1080,16,ds_coarse,None)

(5, 2, 64, 16)
(5, 2, 64, 16)


In [ ]:
plt.plot(time,errors_LI,label="LI")
plt.plot(time,errors_DNS,label="DNS")
plt.xlabel("time (s)")
plt.ylabel("mse")
plt.legend()